<a href="https://colab.research.google.com/github/mahimscit/DO180-apps/blob/master/SQL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL Mandatory Assignment 2

1. Create a user-defined functions to stuff the Chicken into ‘Quick Bites’. Eg: ‘Quick
Chicken Bites’.

In [7]:
import sqlite3

conn = sqlite3.connect('test.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Menu (
        dish_name TEXT
    )
''')
conn.commit()

In [8]:
cursor.execute("INSERT INTO Menu (dish_name) VALUES ('Quick Bites'), ('Burger'), ('Pizza')")
conn.commit()

In [10]:
import sqlite3

conn = sqlite3.connect('test.db')
cursor = conn.cursor()

def stuff_chicken(dish_name):
    """Stuffs 'Chicken' into 'Quick Bites' if present in the dish name."""
    if 'Quick Bites' in dish_name:
        return dish_name.replace('Quick Bites', 'Quick Chicken Bites')
    else:
        return dish_name

conn.create_function('stuff_chicken', 1, stuff_chicken)


cursor.execute("SELECT stuff_chicken(dish_name) FROM Menu")
results = cursor.fetchall()

for row in results:
    print(row[0])

Quick Chicken Bites
Burger
Pizza


2. Use the function to display the restaurant name and cuisine type which has the
maximum number of rating.

In [16]:
import sqlite3

conn = sqlite3.connect('test.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Restaurants (
        name TEXT,
        cuisine TEXT,
        rating_count INTEGER
    )
''')
conn.commit()

restaurants_data = [
    ('Spicy Bites', 'Indian', 1500),
    ('Burger King', 'American', 2000),
    ('Pizza Hut', 'Italian', 1800),
    ('Sushi Heaven', 'Japanese', 1200),
    ('Taco Bell', 'Mexican', 1700)
]

cursor.executemany("INSERT INTO Restaurants (name, cuisine, rating_count) VALUES (?, ?, ?)", restaurants_data)
conn.commit()

cursor.execute('''
    SELECT name, cuisine
    FROM Restaurants
    ORDER BY rating_count DESC
    LIMIT 1
''')

result = cursor.fetchone()

if result:
    restaurant_name, cuisine_type = result
    print(f"Restaurant with maximum ratings: {restaurant_name} ({cuisine_type})")
else:
    print("No restaurants found in the database.")



Restaurant with maximum ratings: Burger King (American)


3. Create a Rating Status column to display the rating as ‘Excellent’ if it has more the 4
start rating,
‘Good’ if it has above 3.5 and below 5 star rating,
and below 3.5 and ‘Bad’ if it is below 3 star rating

In [25]:
import sqlite3

conn = sqlite3.connect('test.db')
cursor = conn.cursor()


update_query = """
UPDATE Restaurants
SET RatingStatus = CASE
    WHEN rating_count > 1900 THEN 'Excellent'
    WHEN rating_count BETWEEN 1600 AND 1900 THEN 'Very Good'
    WHEN rating_count BETWEEN 1200 AND 1599 THEN 'Good'
    WHEN rating_count BETWEEN 800 AND 1199 THEN 'Average'
    ELSE 'Poor'
END;
"""
cursor.execute(update_query)
conn.commit()

cursor.execute("SELECT name, cuisine, RatingStatus FROM Restaurants")
results = cursor.fetchall()

for row in results:
    restaurant_name, cuisine_type, rating_status = row
    print(f"Restaurant: {restaurant_name} ({cuisine_type}), Rating Status: {rating_status}")




Restaurant: Spicy Bites (Indian), Rating Status: Good
Restaurant: Burger King (American), Rating Status: Excellent
Restaurant: Pizza Hut (Italian), Rating Status: Very Good
Restaurant: Sushi Heaven (Japanese), Rating Status: Good
Restaurant: Taco Bell (Mexican), Rating Status: Very Good
Restaurant: Spicy Bites (Indian), Rating Status: Good
Restaurant: Burger King (American), Rating Status: Excellent
Restaurant: Pizza Hut (Italian), Rating Status: Very Good
Restaurant: Sushi Heaven (Japanese), Rating Status: Good
Restaurant: Taco Bell (Mexican), Rating Status: Very Good
Restaurant: Spicy Bites (Indian), Rating Status: Good
Restaurant: Burger King (American), Rating Status: Excellent
Restaurant: Pizza Hut (Italian), Rating Status: Very Good
Restaurant: Sushi Heaven (Japanese), Rating Status: Good
Restaurant: Taco Bell (Mexican), Rating Status: Very Good


4. Find the Ceil, floor and absolute values of the rating column and display the current date
and separately display the year, month_name and day

In [28]:
import sqlite3
import math
from datetime import date

conn = sqlite3.connect('test.db')
cursor = conn.cursor()

cursor.execute("""
SELECT
    rating_count,
    CAST(CEILING(rating_count) AS INT) AS rating_ceil,  -- Ceiling
    CAST(FLOOR(rating_count) AS INT) AS rating_floor,    -- Floor
    ABS(rating_count) AS rating_abs                     -- Absolute
FROM Restaurants;
""")
rating_results = cursor.fetchall()

print("Ceil, Floor, and Absolute Values of Rating:")
for row in rating_results:
    rating_count, rating_ceil, rating_floor, rating_abs = row
    print(f"Rating Count: {rating_count}, Ceil: {rating_ceil}, Floor: {rating_floor}, Absolute: {rating_abs}")

current_date = date.today()
print("\nCurrent Date:", current_date)
print("Year:", current_date.year)
print("Month:", current_date.strftime("%B"))
print("Day:", current_date.day)


Ceil, Floor, and Absolute Values of Rating:
Rating Count: 1500, Ceil: 1500, Floor: 1500, Absolute: 1500
Rating Count: 2000, Ceil: 2000, Floor: 2000, Absolute: 2000
Rating Count: 1800, Ceil: 1800, Floor: 1800, Absolute: 1800
Rating Count: 1200, Ceil: 1200, Floor: 1200, Absolute: 1200
Rating Count: 1700, Ceil: 1700, Floor: 1700, Absolute: 1700
Rating Count: 1500, Ceil: 1500, Floor: 1500, Absolute: 1500
Rating Count: 2000, Ceil: 2000, Floor: 2000, Absolute: 2000
Rating Count: 1800, Ceil: 1800, Floor: 1800, Absolute: 1800
Rating Count: 1200, Ceil: 1200, Floor: 1200, Absolute: 1200
Rating Count: 1700, Ceil: 1700, Floor: 1700, Absolute: 1700
Rating Count: 1500, Ceil: 1500, Floor: 1500, Absolute: 1500
Rating Count: 2000, Ceil: 2000, Floor: 2000, Absolute: 2000
Rating Count: 1800, Ceil: 1800, Floor: 1800, Absolute: 1800
Rating Count: 1200, Ceil: 1200, Floor: 1200, Absolute: 1200
Rating Count: 1700, Ceil: 1700, Floor: 1700, Absolute: 1700

Current Date: 2024-12-17
Year: 2024
Month: December
Day

5. Display the restaurant type and total average cost using rollup.

In [30]:
import sqlite3

conn = sqlite3.connect('test.db')
cursor = conn.cursor()

query = """
SELECT cuisine, AVG(rating_count) AS average_cost
FROM Restaurants
GROUP BY cuisine
"""

cursor.execute(query)
results = cursor.fetchall()

print("Restaurant Type and Total Average Cost (using rating_count):")
for row in results:
    cuisine, average_cost = row
    if cuisine is None:
        print(f"Grand Total Average Cost: {average_cost}")
    else:
        print(f"Cuisine: {cuisine}, Average Cost: {average_cost}")


Restaurant Type and Total Average Cost (using rating_count):
Cuisine: American, Average Cost: 2000.0
Cuisine: Indian, Average Cost: 1500.0
Cuisine: Italian, Average Cost: 1800.0
Cuisine: Japanese, Average Cost: 1200.0
Cuisine: Mexican, Average Cost: 1700.0
